In [1]:
import sys
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib
matplotlib.use('Agg')

from Model import classification_head
from numpy import array
from PIL import ImageOps, Image

WEIGHTS_FILE = "classifier.ckpt"
SIZE = (32,32)

def detect_single(img_path, saved_model_weights,sess):
    X = tf.placeholder(tf.float32, shape=(1, 32, 32, 3))
    prediction = tf.nn.softmax(classification_head(X))

    temp = np.array(img_path)
    reCol = (255-temp)*1.0/255.0
    expanding = np.expand_dims(reCol, axis=0)

    feed_dat = {X: expanding}
    predictions = sess.run(prediction, feed_dict=feed_dat)

    return np.argmax(predictions), temp

In [2]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import glob

image_list = []
saved_model_weights = WEIGHTS_FILE
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, saved_model_weights)
    for filename in glob.glob('Single_Digit_Test/*.png'):
        print("==================================================================================================")
        print("==================================================================================================")
        im=Image.open(filename)
        image_list.append(im)
        im = im.resize(SIZE, Image.ANTIALIAS)
        
        quality_val = 90
        im.save(filename, 'PNG', quality=quality_val)
        
        prediction, test_img = detect_single(im, saved_model_weights,sess)
        plt.figure()
        imgplot = plt.imshow(test_img)
        plt.show()
        print("BEST PREDICTION IS:", prediction)
        print("==================================================================================================")
        print("==================================================================================================")

Populating the interactive namespace from numpy and matplotlib
INFO:tensorflow:Restoring parameters from classifier.ckpt


In [3]:
import matplotlib
matplotlib.use('Agg')

import sys
import os
import numpy as np
import tensorflow as tf
import PIL.Image as Image
import matplotlib.pyplot as plt
import time

from Model import regression_head
from Data_harvest import load_local_svhn_data
from PIL import ImageOps, Image
from tensorflow.python.saved_model import tag_constants
from six.moves import cPickle as pickle
from six.moves import range

WEIGHTS_FILE = "regression.ckpt"
SIZE = (64,64)

def formatNum(pred_array):
    pred_str = ""
    for i in range(len(pred_array)):
        if pred_array[i] != 10:
            pred_str += str(pred_array[i])
        else:
            return pred_str
    return pred_str


def detect_multi(img_path, saved_model_weights ,session):

    print("-------------------Attemped resize img------------------------------")    
    fit_and_resized_image = ImageOps.fit(img_path, SIZE, Image.ANTIALIAS)
    temp = np.array(fit_and_resized_image)
    normalize = (255-temp)*1.0/255.0
    expanding = np.expand_dims(normalize, axis=0)

    X = tf.placeholder(tf.float32, shape=(1, 64, 64, 3))
    [logits_1, logits_2, logits_3, logits_4, logits_5] = regression_head(X)

    predict = tf.stack([tf.nn.softmax(logits_1),
                      tf.nn.softmax(logits_2),
                      tf.nn.softmax(logits_3),
                      tf.nn.softmax(logits_4),
                      tf.nn.softmax(logits_5)])

    best_prediction = tf.transpose(tf.argmax(predict, 2))
    feed_data = {X: expanding}
    start_time = time.time()
    predictions = session.run(best_prediction, feed_dict=feed_data)
    pred = formatNum(predictions[0])
    end_time = time.time()
    return pred, temp

In [4]:
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import glob

WEIGHT_FILE = "regression.ckpt"
image_list = []
saved_model_weights = WEIGHTS_FILE

saver = tf.train.Saver()
with tf.Session() as session:
    saver.restore(session, "regression.ckpt")
    for filename in glob.glob('Multi_Digit_Test/*.png'):
        print("==================================================================================================")
        print("==================================================================================================")
        im=Image.open(filename)
        image_list.append(im)
        prediction, test_img = detect_multi(im, saved_model_weights,session)
        plt.figure()
        imgplot = plt.imshow(test_img)
        plt.show()
        print("BEST PREDICTION IS:", prediction)
        print("==================================================================================================")
        print("==================================================================================================")

Populating the interactive namespace from numpy and matplotlib
INFO:tensorflow:Restoring parameters from regression.ckpt
